<a href="https://colab.research.google.com/github/CryptoGionni/partyQR/blob/main/client_QR_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)                                                       # Indirizzo base del drive
#project_path = '/content/drive/MyDrive/partyQR/data'                               # Path progetto 
#https://drive.google.com/file/d/1IA2OcD389xWAiXHfm9mAcAFIrLSI8orB/view?usp=sharing #link di esempio generato da una immagine-invito-ID

Mounted at /content/drive


#1 genera gli npy con gli HASH ID
eseguilo 3 volte: devi cambiare il nome dell'npy

In [ ]:
pip install qrcode[pil]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 735 kB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=2f87aedfdbe2b571bbcacb4717b3fd2179e7bc958819aa06b07703d70272b2bf
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode


In [ ]:
import uuid         #hash
import random
import string
import numpy as np
import qrcode       #qr

In [ ]:
def hashGenerate(invitato):
    '''
    generate 200 HASH, save them in specific files .npy
    maggiorenni.npy
    minorenni.npy
    premium.npy
        
    ----------
    Parameters
    None.

    ----------
    Returns
    None.

    '''
    hashIds = []#vedi se funziona con sta struttura tupla che non è mutabile https://www.geeksforgeeks.org/python-list-vs-array-vs-tuple/
    
    for i in range(0,200):  #0-199 cioè 200 codici
        # Generate a random string
        # with 32 characters.
        randomstring = ''.join([random.choice(string.ascii_letters
        			+ string.digits) for n in range(32)])
    
        # initializing a string
        ID = str(i) + "passwordSuperSegreta" + randomstring
        
        # using NAMESPACE_URL as namespace
        # to generate SHA-1 hash uuid
        #print (uuid.uuid5(uuid.NAMESPACE_URL, ID))
        
        hashIds.append(uuid.uuid5(uuid.NAMESPACE_URL, ID))
            
    
    np.save('/content/drive/MyDrive/partyQR/data/' + invitato + '.npy', hashIds)

    
    

#2.1 genera il QR dell'ID memorizzato nell'npy
devi eseguirlo 3: cambia l'npy del load e del path in cui salvarli

In [ ]:

    # Printing type of arr object
    #print("Array is of type: ", type(dataIDs))
    
    # Printing array dimensions (axes)
    #print("No. of dimensions: ", dataIDs.ndim)
    
    # Printing shape of array
    #print("Shape of array: ", dataIDs.shape)
    
    # Printing size (total number of elements) of array
    #print("Size of array: ", dataIDs.size)
    
    # Printing type of elements in array
    #print("Array stores elements of type: ", dataIDs.dtype)

    #print(dataIDs[0,]) #corretto

In [ ]:
def saveQr(invitato):


    # Data to be encoded
    dataIDs = np.load("/content/drive/MyDrive/partyQR/data/" + invitato + ".npy", allow_pickle=True)
    
    for i in dataIDs:
      # Encoding data using make() function
     img = qrcode.make(i)  #i anzichè dataIDs[int(i),]
      
      # Saving as an image file
     img.save("/content/drive/MyDrive/partyQR/QrSecco/" + invitato + "/" + 'hashID-' + str(i) + '.png')


#2.2 genera l'immagine invito/locandina e incolla il QR-ID e salvala nella cartella corrispondente

In [ ]:
from PIL import Image
import os


In [ ]:
def meshQR(invitato):
  '''
  unire img background con QR invito
  useful: https://www.geeksforgeeks.org/python-copy-and-paste-images-onto-other-image-using-pillow/
  '''

  # open the image
  template = Image.open('/content/drive/MyDrive/partyQR/template.png')
  QRpath = "/content/drive/MyDrive/partyQR/QrSecco/" + invitato

  # make a copy the image so that
  # the original image does not get affected
  invito = template.copy()
  listImg = os.listdir(QRpath)

  #ha cercato nella cartella giusta (minorenni, premium) una img che ha il nome corrispondente a una img nel path maggiorenni
  #qr_name non viene aggiornata
  for qr_name in listImg:
    qr = Image.open("/content/drive/MyDrive/partyQR/QrSecco/" + invitato + "/" + qr_name)
    Image2copy = qr.copy()

    # paste image giving dimensions
    invito.paste(Image2copy, (355,355))

    # save the image
    invito.save("/content/drive/MyDrive/partyQR/inviti/" + invitato + "/invito-" + qr_name)



# usage: genero gli inviti

In [ ]:
invitati = ['maggiorenni' , 'minorenni' , 'premium']
for chi in invitati:
  #hashGenerate(chi)
  #saveQr(chi)
  #meshQR(chi)



#3.1 - 3.2
3.1 genera link all'invito 
 
3.2 genera QR e salvalo nella corrispondente cartella

In [ ]:
from subprocess import getoutput
from IPython.display import HTML
#mi sa che serve la OS library per prendere i file uno a uno

# need to install xattr
!apt-get install xattr > /dev/null


In [ ]:
def generateLink(invitato):

    
  invitePath = "/content/drive/MyDrive/partyQR/inviti/" + invitato + "/"
  listImg = os.listdir(invitePath)

  for invite_name in listImg:

    #path invito
    invito = invitePath + invite_name

    # get the link
    #https://www.geeksforgeeks.org/string-formatting-in-python/
    fid = getoutput("xattr -p 'user.drive.id' %s "%invito) #questo sarà l'invito effettivo 
        #che contiene il QR-ID. a questo punterà il link qui sotto

    link = "https://drive.google.com/file/d/" + fid + "/view?usp=sharing"

    # Encoding data using make() function
    img = qrcode.make(link)  #0 è il primo elemento del vettore maggiorenni.npy
    
    # Saving as an image file
    img.save('/content/drive/MyDrive/partyQR/link-inviti/link-' + invitato + "/link-" + invite_name)

    #print(fid) #esempio: 1zPTOttz1FWJKt43kdQVNjMX5xZNlWT0O
    #quindi il link saraà del tipo: https://drive.google.com/file/d/1fr0Z1UZxWCiWcF5JRVomitPX4QKVYpzO/view?usp=sharing
    
    #HTML(f"<a href=https://drive.google.com/file/d/{fid}/view?usp=sharing target=_blank>img</a>") #questo diventerà il QR da mostrare 
        #al cliente

#usage

In [ ]:
invitati = ['maggiorenni' , 'minorenni' , 'premium']
for chi in invitati:
  generateLink(chi)